In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import os
import torch
import torch.nn as nn
import torchvision
from torchsummary import summary
from RecursiveUNet3D import UNet3D
from Seg3D import Seg3D
import pandas as pd
import numpy as np
from dataloader import LungDataset_3D_high_res, slab_loader
from torch.utils.data import Dataset, DataLoader
from medpy.io import load,save
import scipy.ndimage

In [3]:
data_path = '/data4/inqlee0704'
in_file = 'ENV18PM_ProjSubjList_cleaned_IN.in'
df_subjlist = pd.read_csv(os.path.join(data_path,in_file),sep='\t')
df_subjlist = df_subjlist[:2]

In [4]:
slabs = slab_loader(df_subjlist)

Loading Data


In [5]:
train_ds = LungDataset_3D_high_res(df_subjlist,slabs)

In [8]:
train_loader = DataLoader(train_ds,
                                batch_size=2,
                                shuffle=False,
                                num_workers=0)

In [11]:
for batch in train_loader:
    img = batch['image']
    mask = batch['seg']
    print(img.shape)
    print(mask.shape)
    
    

torch.Size([2, 1, 256, 256, 32])
torch.Size([2, 1, 256, 256, 32])


In [10]:
print(img.shape)
print(mask.shape)

torch.Size([2, 1, 256, 256, 32])
torch.Size([2, 1, 256, 256, 32])


In [124]:
batch = next(iter(train_ds))

In [60]:
train_ds = LungDataset_3D_high_res(df_subjlist)

In [61]:
batch = next(iter(train_ds))

In [62]:
batch['image'].shape

torch.Size([1, 256, 256, 362])

In [51]:
input = torch.ones(1,1,64,64,64)
Seg = Seg3D()


In [63]:
summary(Seg.to('cuda'),(1,512,512,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1     [-1, 32, 256, 256, 32]             896
       BatchNorm3d-2     [-1, 32, 256, 256, 32]              64
              ReLU-3     [-1, 32, 256, 256, 32]               0
            Conv3d-4     [-1, 32, 256, 256, 32]          27,680
       BatchNorm3d-5     [-1, 32, 256, 256, 32]              64
              ReLU-6     [-1, 32, 256, 256, 32]               0
            Conv3d-7      [-1, 1, 256, 256, 32]              33
         MaxPool3d-8     [-1, 32, 128, 128, 16]               0
            Conv3d-9     [-1, 64, 128, 128, 16]          55,360
      BatchNorm3d-10     [-1, 64, 128, 128, 16]             128
             ReLU-11     [-1, 64, 128, 128, 16]               0
           Conv3d-12     [-1, 64, 128, 128, 16]         110,656
      BatchNorm3d-13     [-1, 64, 128, 128, 16]             128
             ReLU-14     [-1, 64, 128, 

In [12]:
UNet = UNet3D(num_classes=1,num_downs=3)
summary(UNet.to('cuda'),(1,128,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1    [-1, 64, 128, 128, 128]           1,792
    InstanceNorm3d-2    [-1, 64, 128, 128, 128]               0
         LeakyReLU-3    [-1, 64, 128, 128, 128]               0
            Conv3d-4    [-1, 64, 128, 128, 128]         110,656
    InstanceNorm3d-5    [-1, 64, 128, 128, 128]               0
         LeakyReLU-6    [-1, 64, 128, 128, 128]               0
         MaxPool3d-7       [-1, 64, 64, 64, 64]               0
            Conv3d-8      [-1, 128, 64, 64, 64]         221,312
    InstanceNorm3d-9      [-1, 128, 64, 64, 64]               0
        LeakyReLU-10      [-1, 128, 64, 64, 64]               0
           Conv3d-11      [-1, 128, 64, 64, 64]         442,496
   InstanceNorm3d-12      [-1, 128, 64, 64, 64]               0
        LeakyReLU-13      [-1, 128, 64, 64, 64]               0
        MaxPool3d-14      [-1, 128, 32,

In [8]:
out_seg = Seg(input.to('cuda'))
out_seg.shape

torch.Size([1, 2, 64, 64, 64])

In [9]:
out_UNet = UNet(input.to('cuda'))
out_UNet.shape

torch.Size([1, 1, 64, 64, 64])